In [ ]:
#!pip install openai

### Pre-defined Method

In [1]:
import json
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import re

In [2]:
import re

SPECIALS = "".join([',', '.'])  # 허용할 특수 문자

SPECIALS = re.escape(SPECIALS)

def preprocess(text: str):
    
    # 특수 문자, 숫자만 남기기
    text = re.sub(f"[^0-9{SPECIALS}]", "", text)  # f-문자열을 적용시켜 SPECIALS를 넣어줌.
    
    # 연속 공백 제거
    text = re.sub(" +", " ", text)
    
    # 좌우 불필요한 공백 제거
    return text.strip()

In [3]:
from sklearn.metrics import mean_absolute_error
import numpy as np

def test_compute_metrics(label, pred):
    MAE = []
    
    for i in range(5):
        if pred[i][-1] == '.':
            pred[i] = pred[i][:-1]
        
        MAE.append(1 - mean_absolute_error([float(label[i])], [float(pred[i])]))
    
    
    return MAE

### Zero-shot LLM Inference

In [ ]:
"""
You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323]
"""

In [19]:
key_path = "/home/egg2018037024/juphome/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['Script'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
        #    top_p = 0.92
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

100%|██████████| 1970/1970 [25:03<00:00,  1.31it/s]


In [16]:
result_df['Inference'].to_list()

['[O: 0.4875, C: 0.5320, E: 0.6532, A: 0.4218, N: 0.7543]',
 'Based on the given sentence, here are the scores for the OCEAN personality traits:\n\n[O: 0.4556, C: 0.4212, E: 0.5898, A: 0.2789, N: 0.7567]',
 '[O: 0.6798, C: 0.7456, E: 0.5673, A: 0.8234, N: 0.4212]',
 '[O: 0.7564, C: 0.6891, E: 0.5323, A: 0.7856, N: 0.4210]',
 '[O: 0.8775, C: 0.6821, E: 0.4213, A: 0.7536, N: 0.4912]',
 '[O: 0.8543, C: 0.7567, E: 0.4211, A: 0.9132, N: 0.3256]',
 '[O: 0.8456, C: 0.7423, E: 0.5721, A: 0.8139, N: 0.3124]',
 '0.722, C: 0.839, E: 0.467, A: 0.521, N: 0.624]',
 'Based on the given sentence, the scores for the OCEAN personality traits are as follows:\n\nO (Openness): 0.7320\nC (Conscientiousness): 0.2543\nE (Extraversion): 0.1287\nA (Agreeableness): 0.4891\nN (Neuroticism): 0.7135',
 '[O: 0.2134, C: 0.6647, E: 0.2156, A: 0.7682, N: 0.7259]',
 '[O: 0.7323, C: 0.5845, E: 0.4768, A: 0.8176, N: 0.6234]',
 'O: 0.6785, C: 0.7321, E: 0.7896, A: 0.8452, N: 0.4210',
 '[O: 0.6321, C: 0.7450, E: 0.5213, A: 

In [37]:
result_df['Inference'].to_list()[1624]  # Error Example

'I apologize, but it seems the Korean text provided is not suitable for analysis. If you have an English sentence or any text in a language I can process, please feel free to share it for the personality trait analysis.'

In [28]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [29]:
score

[['0.4875', '0.5320', '0.6532', '0.4218', '0.7543'],
 ['0.4556', '0.4212', '0.5898', '0.2789', '0.7567'],
 ['0.6798', '0.7456', '0.5673', '0.8234', '0.4212'],
 ['0.7564', '0.6891', '0.5323', '0.7856', '0.4210'],
 ['0.8775', '0.6821', '0.4213', '0.7536', '0.4912'],
 ['0.8543', '0.7567', '0.4211', '0.9132', '0.3256'],
 ['0.8456', '0.7423', '0.5721', '0.8139', '0.3124'],
 ['0.722', '0.839', '0.467', '0.521', '0.624'],
 ['0.7320', '0.2543', '0.1287', '0.4891', '0.7135'],
 ['0.2134', '0.6647', '0.2156', '0.7682', '0.7259'],
 ['0.7323', '0.5845', '0.4768', '0.8176', '0.6234'],
 ['0.6785', '0.7321', '0.7896', '0.8452', '0.4210'],
 ['0.6321', '0.7450', '0.5213', '0.8124', '0.6321'],
 ['0.7567', '0.7234', '0.6789', '0.8123', '0.4566'],
 ['0.1234', '0.5678', '0.3456', '0.4321', '0.7890'],
 ['0.8333', '0.6667', '0.5', '0.8333', '0.5'],
 ['0.5465', '0.7238', '0.4192', '0.6247', '0.3661'],
 ['0.7455', '0.6783', '0.4567', '0.8234', '0.3211'],
 ['0.2513', '0.6138', '0.4979', '0.7521', '0.8496'],
 ['0

In [26]:
FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv",
                     index_col=0)
FI_test['label'] = FI_test[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_test['label'] = FI_test['label'].str.split(' ')

FI_test

,id,Script,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness,token_len,user,l_Script,label
0,htH89DBizno.004.mp4,다른 방에서 미친 듯이 달려가서 없는지 확인했는데 없는 것 같았어요 이제 잠을 충분...,0.485981,0.645833,0.681319,0.669903,0.626168,0.822222,36,htH89DBiznomp4,going nuts from another room run in there to c...,"[0.8222222222222221, 0.6699029126213591, 0.485..."
1,p_wf-KszNlk.001.mp4,갈 길이 조금 남았지만 당신이 필요해서 Q&A 비디오를하기로 결정했습니다. 분명히 ...,0.616822,0.593750,0.692308,0.514563,0.570093,0.655556,40,p_wf-KszNlkmp4,i've got a little bit to go but we need you th...,"[0.6555555555555556, 0.5145631067961165, 0.616..."
2,MuYYY3XaJ7Q.001.mp4,영상은 양보다 질이 중요하기 때문에 모든 사람이 볼 수 있는 기회를 제공하기 위해 ...,0.467290,0.625000,0.560440,0.524272,0.514019,0.522222,50,MuYYY3XaJ7Qmp4,a video's quality over quantity so everyone ca...,"[0.5222222222222223, 0.5242718446601942, 0.467..."
3,0MB91ku0eEw.005.mp4,"제가 한 일은 온라인에서 사람들을 가상으로 만났을 때 그들이 게시하는 것을보고 ""저...",0.411215,0.458333,0.714286,0.660194,0.570093,0.400000,53,0MB91ku0eEwmp4,you really want to that's made things a lot ea...,"[0.4, 0.6601941747572816, 0.4112149532710281, ..."
4,WpEZOSrENL0.003.mp4,이 연습의 요점은 실제로 답을 찾는 것이 아닙니다. 요점은 단순히 이러한 질문을 스...,0.317757,0.437500,0.384615,0.524272,0.448598,0.411111,39,WpEZOSrENL0mp4,that's perfectly okay the point isn't actually...,"[0.4111111111111111, 0.5242718446601942, 0.317..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,NzCflfxgvwo.001.mp4,두두두 아니 그게 내가이 비디오를 만든 이유가 아니야 나는이 비디오를 만들었다 나는...,0.289720,0.312500,0.208791,0.135922,0.177570,0.300000,49,NzCflfxgvwomp4,du du du no that's not why i made this video i...,"[0.3, 0.1359223300970873, 0.2897196261682243, ..."
1996,ch2BcBv4SdQ.003.mp4,그들은 항상 그렇게 합니다.,0.719626,0.781250,0.670330,0.572816,0.728972,0.722222,6,ch2BcBv4SdQmp4,they do it all the time,"[0.7222222222222221, 0.5728155339805825, 0.719..."
1997,It0-yCwOc5k.001.mp4,편안하고 누구도 불편함을 느끼지 않기를 바랍니다 그래서 내 채널에서 다른 사람들을 ...,0.355140,0.395833,0.472527,0.446602,0.411215,0.677778,30,It0-yCwOc5kmp4,comfortable and i don't want anyone to feel un...,"[0.6777777777777777, 0.4466019417475728, 0.355..."
1998,1Wh2mIFrGvk.000.mp4,새로운 근육을 만들기에 충분한 칼로리를 공급하지 못하고 있습니다 새로운 조직은 어디...,0.467290,0.645833,0.527473,0.669903,0.560748,0.622222,44,1Wh2mIFrGvkmp4,you're not giving yourself enough calories to ...,"[0.6222222222222222, 0.6699029126213591, 0.467..."


In [38]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

317 list index out of range
552 list index out of range
1288 list index out of range
1436 list index out of range
1624 list index out of range
1859 list index out of range


In [39]:
error_idx

[317, 552, 1288, 1436, 1624, 1859]

In [41]:
re_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['Script'][error_idx].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323]",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
        #    top_p = 0.92
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    re_df = pd.concat([re_df, tmp_df])

100%|██████████| 6/6 [00:09<00:00,  1.63s/it]


In [42]:
re_df

,Text,Inference
0,2016년 새해 복 많이 받으세요 올해가 우리 모두에게 어떤 해가 될지 생각하면 너...,"[O: 0.8333, C: 0.7222, E: 0.6667, A: 0.7222, N..."
0,내가 정말 할 수 있는 일이 없을 것 같아서 줄 만한 가치 있는 것 칵테일 셰이커나...,"0.7663, C: 0.2121, E: 0.4545, A: 0.3636, N: 0...."
0,물론 당신은 전에 보았는지 모르겠지만 지금은 내 차에 있지만 나중에 보여줄 것입니다...,"[O: 0.1220, C: 0.7543, E: 0.2651, A: 0.4198, N..."
0,제가 책에 넣은 질문에 답해주시길 바랍니다.,"0.8333, C: 0.7222, E: 0.3889, A: 0.8333, N: 0...."
0,나보다 잘생겼거나 나보다 못생겼거나 마음속에서 자신감을 가져야 하는 이유에 대해 의...,"[O: 0.7642, C: 0.6780, E: 0.3256, A: 0.7521, N..."
0,"좋은 사랑스러운 감성 책 그래서 네, 그렇게 말해야 할 것입니다 다음 질문 ""가장 ...","[O: 0.8765, C: 0.3456, E: 0.5678, A: 0.7345, N..."


In [43]:
re_score = []
p = re.compile('0[.]\d*')

for content in re_df['Inference'].to_list():
    tmp = p.findall(content)
    re_score.append(tmp)

In [44]:
re_score

[['0.8333', '0.7222', '0.6667', '0.7222', '0.4444'],
 ['0.7663', '0.2121', '0.4545', '0.3636', '0.8571'],
 ['0.1220', '0.7543', '0.2651', '0.4198', '0.6212'],
 ['0.8333', '0.7222', '0.3889', '0.8333', '0.2778'],
 ['0.7642', '0.6780', '0.3256', '0.7521', '0.4198'],
 ['0.8765', '0.3456', '0.5678', '0.7345', '0.4567']]

In [46]:
for i, idx in enumerate(error_idx):
    score[idx] = re_score[i]

In [47]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [48]:
np.array(MAE_arr).mean(axis=0)

array([0.77591929, 0.80950776, 0.82529518, 0.82160068, 0.80991879])

In [49]:
np.mean([0.77591929, 0.80950776, 0.82529518, 0.82160068, 0.80991879])

0.80844834

--------------------

### One-shot LLM Inference

In [50]:
key_path = "/home/egg2018037024/juphome/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

one_shot_text = "현재 가장 좋아하는 책 사실 몇 달 전에 서점에서 이 책을 샀는데 읽을 기회가 없었는데 읽기 시작했어요 한번 시작하면 내려놓고 싶지 않은 책 조이스 마이어의 <용감하게 살기>라는 책이에요."

for data in tqdm(FI_test['Script'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + one_shot_text + '",\n Results- [O: 0.6333, C: 0.7184, E: 0.4019, A: 0.4176, N: 0.5313] \n'
            },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
        #    top_p = 0.92
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

100%|██████████| 1970/1970 [33:50<00:00,  1.03s/it]


In [51]:
result_df['Inference'].to_list()

['0.3456, C: 0.5782, E: 0.6891, A: 0.2674, N: 0.7523]',
 'Results- [O: 0.3112, C: 0.3276, E: 0.4667, A: 0.2554, N: 0.8111]',
 '0.7621, C: 0.8495, E: 0.4308, A: 0.7592, N: 0.3196]',
 '0.6712, C: 0.7521, E: 0.4895, A: 0.6243, N: 0.4568]',
 '0.7778, C: 0.7667, E: 0.4333, A: 0.6333, N: 0.4222]',
 '0.6882, C: 0.7826, E: 0.4888, A: 0.6163, N: 0.3567]',
 'Results: [O: 0.6542, C: 0.7581, E: 0.6137, A: 0.8152, N: 0.4069]',
 '0.7442, C: 0.6938, E: 0.3651, A: 0.2829, N: 0.6183]',
 '0.4667, C: 0.6222, E: 0.3667, A: 0.5111, N: 0.5444]',
 '0.3942, C: 0.6012, E: 0.2648, A: 0.1845, N: 0.6973]',
 '[O: 0.6642, C: 0.6049, E: 0.5781, A: 0.7606, N: 0.4388]',
 '0.4978, C: 0.6245, E: 0.7312, A: 0.6843, N: 0.3701]',
 '0.5943, C: 0.7601, E: 0.5928, A: 0.6947, N: 0.4489]',
 'Given Sentence: "영상에서 대답할 수 없는 질문의 스크린샷을 찍었기 때문에 누구의 질문인지 알려드립니다. "왜 헤어졌고, 대부분 남자 또는 여자에게 더 끌리나요?"라는 질문입니다. 라이언과 함께하고있는 비디오에서이 질문에 답할 것입니다."\nResults- [O: 0.3489, C: 0.7256, E: 0.3765, A: 0.6211, N: 0.5413]',
 'Results: [O: 0.3894, C: 0.516

In [52]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [53]:
score

[['0.3456', '0.5782', '0.6891', '0.2674', '0.7523'],
 ['0.3112', '0.3276', '0.4667', '0.2554', '0.8111'],
 ['0.7621', '0.8495', '0.4308', '0.7592', '0.3196'],
 ['0.6712', '0.7521', '0.4895', '0.6243', '0.4568'],
 ['0.7778', '0.7667', '0.4333', '0.6333', '0.4222'],
 ['0.6882', '0.7826', '0.4888', '0.6163', '0.3567'],
 ['0.6542', '0.7581', '0.6137', '0.8152', '0.4069'],
 ['0.7442', '0.6938', '0.3651', '0.2829', '0.6183'],
 ['0.4667', '0.6222', '0.3667', '0.5111', '0.5444'],
 ['0.3942', '0.6012', '0.2648', '0.1845', '0.6973'],
 ['0.6642', '0.6049', '0.5781', '0.7606', '0.4388'],
 ['0.4978', '0.6245', '0.7312', '0.6843', '0.3701'],
 ['0.5943', '0.7601', '0.5928', '0.6947', '0.4489'],
 ['0.3489', '0.7256', '0.3765', '0.6211', '0.5413'],
 ['0.3894', '0.5167', '0.3502', '0.2736', '0.7669'],
 ['0.7465', '0.6638', '0.5782', '0.7389', '0.4221'],
 ['0.5111', '0.7555', '0.4444', '0.6555', '0.4333'],
 ['0.6154', '0.7692', '0.5769', '0.8077', '0.4231'],
 ['0.3541', '0.6110', '0.5498', '0.7202', '0.8

In [54]:
FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv",
                     index_col=0)
FI_test['label'] = FI_test[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_test['label'] = FI_test['label'].str.split(' ')

FI_test

,id,Script,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness,token_len,user,l_Script,label
0,htH89DBizno.004.mp4,다른 방에서 미친 듯이 달려가서 없는지 확인했는데 없는 것 같았어요 이제 잠을 충분...,0.485981,0.645833,0.681319,0.669903,0.626168,0.822222,36,htH89DBiznomp4,going nuts from another room run in there to c...,"[0.8222222222222221, 0.6699029126213591, 0.485..."
1,p_wf-KszNlk.001.mp4,갈 길이 조금 남았지만 당신이 필요해서 Q&A 비디오를하기로 결정했습니다. 분명히 ...,0.616822,0.593750,0.692308,0.514563,0.570093,0.655556,40,p_wf-KszNlkmp4,i've got a little bit to go but we need you th...,"[0.6555555555555556, 0.5145631067961165, 0.616..."
2,MuYYY3XaJ7Q.001.mp4,영상은 양보다 질이 중요하기 때문에 모든 사람이 볼 수 있는 기회를 제공하기 위해 ...,0.467290,0.625000,0.560440,0.524272,0.514019,0.522222,50,MuYYY3XaJ7Qmp4,a video's quality over quantity so everyone ca...,"[0.5222222222222223, 0.5242718446601942, 0.467..."
3,0MB91ku0eEw.005.mp4,"제가 한 일은 온라인에서 사람들을 가상으로 만났을 때 그들이 게시하는 것을보고 ""저...",0.411215,0.458333,0.714286,0.660194,0.570093,0.400000,53,0MB91ku0eEwmp4,you really want to that's made things a lot ea...,"[0.4, 0.6601941747572816, 0.4112149532710281, ..."
4,WpEZOSrENL0.003.mp4,이 연습의 요점은 실제로 답을 찾는 것이 아닙니다. 요점은 단순히 이러한 질문을 스...,0.317757,0.437500,0.384615,0.524272,0.448598,0.411111,39,WpEZOSrENL0mp4,that's perfectly okay the point isn't actually...,"[0.4111111111111111, 0.5242718446601942, 0.317..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,NzCflfxgvwo.001.mp4,두두두 아니 그게 내가이 비디오를 만든 이유가 아니야 나는이 비디오를 만들었다 나는...,0.289720,0.312500,0.208791,0.135922,0.177570,0.300000,49,NzCflfxgvwomp4,du du du no that's not why i made this video i...,"[0.3, 0.1359223300970873, 0.2897196261682243, ..."
1996,ch2BcBv4SdQ.003.mp4,그들은 항상 그렇게 합니다.,0.719626,0.781250,0.670330,0.572816,0.728972,0.722222,6,ch2BcBv4SdQmp4,they do it all the time,"[0.7222222222222221, 0.5728155339805825, 0.719..."
1997,It0-yCwOc5k.001.mp4,편안하고 누구도 불편함을 느끼지 않기를 바랍니다 그래서 내 채널에서 다른 사람들을 ...,0.355140,0.395833,0.472527,0.446602,0.411215,0.677778,30,It0-yCwOc5kmp4,comfortable and i don't want anyone to feel un...,"[0.6777777777777777, 0.4466019417475728, 0.355..."
1998,1Wh2mIFrGvk.000.mp4,새로운 근육을 만들기에 충분한 칼로리를 공급하지 못하고 있습니다 새로운 조직은 어디...,0.467290,0.645833,0.527473,0.669903,0.560748,0.622222,44,1Wh2mIFrGvkmp4,you're not giving yourself enough calories to ...,"[0.6222222222222222, 0.6699029126213591, 0.467..."


In [55]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

980 list index out of range


In [56]:
error_idx

[980]

In [57]:
re_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['Script'][error_idx].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323]",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
        #    top_p = 0.92
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    re_df = pd.concat([re_df, tmp_df])

100%|██████████| 1/1 [00:06<00:00,  6.37s/it]


In [58]:
re_df

,Text,Inference
0,보통 편집이 많이 걸리는 당일에 보통 꽤 오래 걸립니다 크리스마스처럼 마지막으로 한...,"O: 0.1212, C: 0.8789, E: 0.0904, A: 0.5678, N:..."


In [59]:
re_score = []
p = re.compile('0[.]\d*')

for content in re_df['Inference'].to_list():
    tmp = p.findall(content)
    re_score.append(tmp)

In [60]:
re_score

[['0.1212', '0.8789', '0.0904', '0.5678', '0.7789']]

In [61]:
for i, idx in enumerate(error_idx):
    score[idx] = re_score[i]

In [62]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [63]:
np.array(MAE_arr).mean(axis=0)

array([0.84711233, 0.81414687, 0.83802557, 0.83684268, 0.8280393 ])

In [65]:
np.mean([0.84711233, 0.81414687, 0.83802557, 0.83684268, 0.8280393])

0.8328333500000001

--------------------

### Five-Shot Inference

In [50]:
key_path = "/home/egg2018037024/juphome/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

few_shot_text0 = "현재 가장 좋아하는 책 사실 몇 달 전에 서점에서 이 책을 샀는데 읽을 기회가 없었는데 읽기 시작했어요 한번 시작하면 내려놓고 싶지 않은 책 조이스 마이어의 <용감하게 살기>라는 책이에요."
few_shot_text1 = "제가 타겟에서 일했던 곳과 월마트에서 일했던 곳 제가 유튜브를하지 않았다면 아마 여전히 그 일을하고 있지 않았을 것입니다. 저는 항상 손으로 일하고 물건을 만드는 데 관심이 있었지만 건설은 그랬어요"
few_shot_text2 = "나쁘지만 그들이 만든 최악의 영화는 아닙니다. 그들이 만든 최악의 영화는 괴물이라고 생각합니다. 괴물은 텍사스의 비료 판매원이 만들었다는 점에서 정말 흥미 롭습니다."
few_shot_text3 = "매우 소외감을 느낀다는 것은 확실히 제 자존감과 제 자신을 보는 방식에 타격을 주었습니다 제가 이것을 극복한 방법은 제가 매우 긍정적인 환경에 있는지 확인했습니다 저는 합창단처럼 제가 좋아하는 일을 계속하고 예술을 만들었습니다."
few_shot_text4 = "감정을 읽는 것에 대해 사람들의 얼굴을 보고 그들의 감정을 어떻게 알 수 있는지에 관한 것입니다 흥미롭습니다 나는 유령 요금소를 읽을 것입니다 나는 그것을 읽지 않았다 나는 많은 사람들이 가지고 있다는 것을 알고 있습니다 미안합니다 나는 그것을 읽지 않았다."


for data in tqdm(FI_test['Script'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text0 + '",\n Results- [O: 0.6333, C: 0.7184, E: 0.4019, A: 0.4176, N: 0.5313] \n'
            },
       {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text1 + '",\n Results- [O: 0.6889, C: 0.5631, E: 0.5981, A: 0.6374, N: 0.6563] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text2 + '",\n Results- [O: 0.7778, C: 0.4175, E: 0.5794, A: 0.6813, N: 0.65625] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text3 + '",\n Results- [O: 0.9000, C: 0.8835, E: 0.7103, A: 0.7912, N: 0.8229] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text4 + '",\n Results- [O: 0.3889, C: 0.4660, E: 0.2056, A: 0.4176, N: 0.3646] \n'
            },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]

    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
        #    top_p = 0.92
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

100%|██████████| 1970/1970 [33:50<00:00,  1.03s/it]


In [51]:
result_df['Inference'].to_list()

['0.3456, C: 0.5782, E: 0.6891, A: 0.2674, N: 0.7523]',
 'Results- [O: 0.3112, C: 0.3276, E: 0.4667, A: 0.2554, N: 0.8111]',
 '0.7621, C: 0.8495, E: 0.4308, A: 0.7592, N: 0.3196]',
 '0.6712, C: 0.7521, E: 0.4895, A: 0.6243, N: 0.4568]',
 '0.7778, C: 0.7667, E: 0.4333, A: 0.6333, N: 0.4222]',
 '0.6882, C: 0.7826, E: 0.4888, A: 0.6163, N: 0.3567]',
 'Results: [O: 0.6542, C: 0.7581, E: 0.6137, A: 0.8152, N: 0.4069]',
 '0.7442, C: 0.6938, E: 0.3651, A: 0.2829, N: 0.6183]',
 '0.4667, C: 0.6222, E: 0.3667, A: 0.5111, N: 0.5444]',
 '0.3942, C: 0.6012, E: 0.2648, A: 0.1845, N: 0.6973]',
 '[O: 0.6642, C: 0.6049, E: 0.5781, A: 0.7606, N: 0.4388]',
 '0.4978, C: 0.6245, E: 0.7312, A: 0.6843, N: 0.3701]',
 '0.5943, C: 0.7601, E: 0.5928, A: 0.6947, N: 0.4489]',
 'Given Sentence: "영상에서 대답할 수 없는 질문의 스크린샷을 찍었기 때문에 누구의 질문인지 알려드립니다. "왜 헤어졌고, 대부분 남자 또는 여자에게 더 끌리나요?"라는 질문입니다. 라이언과 함께하고있는 비디오에서이 질문에 답할 것입니다."\nResults- [O: 0.3489, C: 0.7256, E: 0.3765, A: 0.6211, N: 0.5413]',
 'Results: [O: 0.3894, C: 0.516

In [66]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [67]:
score

[['0.3456', '0.5782', '0.6891', '0.2674', '0.7523'],
 ['0.3112', '0.3276', '0.4667', '0.2554', '0.8111'],
 ['0.7621', '0.8495', '0.4308', '0.7592', '0.3196'],
 ['0.6712', '0.7521', '0.4895', '0.6243', '0.4568'],
 ['0.7778', '0.7667', '0.4333', '0.6333', '0.4222'],
 ['0.6882', '0.7826', '0.4888', '0.6163', '0.3567'],
 ['0.6542', '0.7581', '0.6137', '0.8152', '0.4069'],
 ['0.7442', '0.6938', '0.3651', '0.2829', '0.6183'],
 ['0.4667', '0.6222', '0.3667', '0.5111', '0.5444'],
 ['0.3942', '0.6012', '0.2648', '0.1845', '0.6973'],
 ['0.6642', '0.6049', '0.5781', '0.7606', '0.4388'],
 ['0.4978', '0.6245', '0.7312', '0.6843', '0.3701'],
 ['0.5943', '0.7601', '0.5928', '0.6947', '0.4489'],
 ['0.3489', '0.7256', '0.3765', '0.6211', '0.5413'],
 ['0.3894', '0.5167', '0.3502', '0.2736', '0.7669'],
 ['0.7465', '0.6638', '0.5782', '0.7389', '0.4221'],
 ['0.5111', '0.7555', '0.4444', '0.6555', '0.4333'],
 ['0.6154', '0.7692', '0.5769', '0.8077', '0.4231'],
 ['0.3541', '0.6110', '0.5498', '0.7202', '0.8

In [68]:
FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv",
                     index_col=0)
FI_test['label'] = FI_test[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_test['label'] = FI_test['label'].str.split(' ')

FI_test

,id,Script,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness,token_len,user,l_Script,label
0,htH89DBizno.004.mp4,다른 방에서 미친 듯이 달려가서 없는지 확인했는데 없는 것 같았어요 이제 잠을 충분...,0.485981,0.645833,0.681319,0.669903,0.626168,0.822222,36,htH89DBiznomp4,going nuts from another room run in there to c...,"[0.8222222222222221, 0.6699029126213591, 0.485..."
1,p_wf-KszNlk.001.mp4,갈 길이 조금 남았지만 당신이 필요해서 Q&A 비디오를하기로 결정했습니다. 분명히 ...,0.616822,0.593750,0.692308,0.514563,0.570093,0.655556,40,p_wf-KszNlkmp4,i've got a little bit to go but we need you th...,"[0.6555555555555556, 0.5145631067961165, 0.616..."
2,MuYYY3XaJ7Q.001.mp4,영상은 양보다 질이 중요하기 때문에 모든 사람이 볼 수 있는 기회를 제공하기 위해 ...,0.467290,0.625000,0.560440,0.524272,0.514019,0.522222,50,MuYYY3XaJ7Qmp4,a video's quality over quantity so everyone ca...,"[0.5222222222222223, 0.5242718446601942, 0.467..."
3,0MB91ku0eEw.005.mp4,"제가 한 일은 온라인에서 사람들을 가상으로 만났을 때 그들이 게시하는 것을보고 ""저...",0.411215,0.458333,0.714286,0.660194,0.570093,0.400000,53,0MB91ku0eEwmp4,you really want to that's made things a lot ea...,"[0.4, 0.6601941747572816, 0.4112149532710281, ..."
4,WpEZOSrENL0.003.mp4,이 연습의 요점은 실제로 답을 찾는 것이 아닙니다. 요점은 단순히 이러한 질문을 스...,0.317757,0.437500,0.384615,0.524272,0.448598,0.411111,39,WpEZOSrENL0mp4,that's perfectly okay the point isn't actually...,"[0.4111111111111111, 0.5242718446601942, 0.317..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,NzCflfxgvwo.001.mp4,두두두 아니 그게 내가이 비디오를 만든 이유가 아니야 나는이 비디오를 만들었다 나는...,0.289720,0.312500,0.208791,0.135922,0.177570,0.300000,49,NzCflfxgvwomp4,du du du no that's not why i made this video i...,"[0.3, 0.1359223300970873, 0.2897196261682243, ..."
1996,ch2BcBv4SdQ.003.mp4,그들은 항상 그렇게 합니다.,0.719626,0.781250,0.670330,0.572816,0.728972,0.722222,6,ch2BcBv4SdQmp4,they do it all the time,"[0.7222222222222221, 0.5728155339805825, 0.719..."
1997,It0-yCwOc5k.001.mp4,편안하고 누구도 불편함을 느끼지 않기를 바랍니다 그래서 내 채널에서 다른 사람들을 ...,0.355140,0.395833,0.472527,0.446602,0.411215,0.677778,30,It0-yCwOc5kmp4,comfortable and i don't want anyone to feel un...,"[0.6777777777777777, 0.4466019417475728, 0.355..."
1998,1Wh2mIFrGvk.000.mp4,새로운 근육을 만들기에 충분한 칼로리를 공급하지 못하고 있습니다 새로운 조직은 어디...,0.467290,0.645833,0.527473,0.669903,0.560748,0.622222,44,1Wh2mIFrGvkmp4,you're not giving yourself enough calories to ...,"[0.6222222222222222, 0.6699029126213591, 0.467..."


In [69]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

980 list index out of range


In [70]:
error_idx

[980]

In [75]:
re_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['Script'][error_idx].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323]",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
        #    top_p = 0.92
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    re_df = pd.concat([re_df, tmp_df])

100%|██████████| 1/1 [00:03<00:00,  3.98s/it]


In [81]:
re_df

,Text,Inference
0,보통 편집이 많이 걸리는 당일에 보통 꽤 오래 걸립니다 크리스마스처럼 마지막으로 한...,"[O: 0.1634, C: 0.7501, E: 0.3865, A: 0.5887, N..."


In [82]:
re_score = []
p = re.compile('0[.]\d*')

for content in re_df['Inference'].to_list():
    tmp = p.findall(content)
    re_score.append(tmp)

In [83]:
re_score

[['0.1634', '0.7501', '0.3865', '0.5887', '0.4219']]

In [84]:
for i, idx in enumerate(error_idx):
    score[idx] = re_score[i]

In [85]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [86]:
np.array(MAE_arr).mean(axis=0)

array([0.84713375, 0.81421225, 0.83817587, 0.83683208, 0.82817295])

In [88]:
np.mean([0.84713375, 0.81421225, 0.83817587, 0.83683208, 0.82817295])

0.83290538

### GPT4 Inference - Zero Shot

In [91]:
key_path = "/home/egg2018037024/juphome/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['Script'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages,
            temperature=0.9,
        #    top_p = 0.92
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

100%|██████████| 1970/1970 [3:12:14<00:00,  5.86s/it]  


In [16]:
result_df['Inference'].to_list()

['[O: 0.4875, C: 0.5320, E: 0.6532, A: 0.4218, N: 0.7543]',
 'Based on the given sentence, here are the scores for the OCEAN personality traits:\n\n[O: 0.4556, C: 0.4212, E: 0.5898, A: 0.2789, N: 0.7567]',
 '[O: 0.6798, C: 0.7456, E: 0.5673, A: 0.8234, N: 0.4212]',
 '[O: 0.7564, C: 0.6891, E: 0.5323, A: 0.7856, N: 0.4210]',
 '[O: 0.8775, C: 0.6821, E: 0.4213, A: 0.7536, N: 0.4912]',
 '[O: 0.8543, C: 0.7567, E: 0.4211, A: 0.9132, N: 0.3256]',
 '[O: 0.8456, C: 0.7423, E: 0.5721, A: 0.8139, N: 0.3124]',
 '0.722, C: 0.839, E: 0.467, A: 0.521, N: 0.624]',
 'Based on the given sentence, the scores for the OCEAN personality traits are as follows:\n\nO (Openness): 0.7320\nC (Conscientiousness): 0.2543\nE (Extraversion): 0.1287\nA (Agreeableness): 0.4891\nN (Neuroticism): 0.7135',
 '[O: 0.2134, C: 0.6647, E: 0.2156, A: 0.7682, N: 0.7259]',
 '[O: 0.7323, C: 0.5845, E: 0.4768, A: 0.8176, N: 0.6234]',
 'O: 0.6785, C: 0.7321, E: 0.7896, A: 0.8452, N: 0.4210',
 '[O: 0.6321, C: 0.7450, E: 0.5213, A: 

In [92]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [93]:
score

[['0.4', '0.2', '0.3', '0.3', '0.7'],
 ['0.65', '0.35', '0.75', '0.30', '0.85'],
 ['0.62', '0.75', '0.58', '0.85', '0.25'],
 ['0.630', '0.310', '0.580', '0.450', '0.200'],
 ['0.765', '0.412', '0.248', '0.580', '0.319'],
 ['0.6', '0.2', '0.4', '0.7', '0.3'],
 ['0.7500', '0.4100', '0.8200', '0.6500', '0.2000'],
 ['0.75', '0.65', '0.35', '0.40', '0.55'],
 ['0.55', '0.10', '0.30', '0.40', '0.70'],
 ['0.2', '0.1', '0.3', '0.2', '0.7'],
 ['0.7', '0.4', '0.8', '0.9', '0.6', '0.7', '0.4', '0.8', '0.9', '0.6'],
 ['0.6', '0.2', '0.5', '0.7', '0.3'],
 ['0.4', '0.7', '0.6', '0.8', '0.3'],
 ['0.3', '0.2', '0.55', '0.4', '0.1'],
 ['0.2', '0.1', '0.3', '0.1', '0.7'],
 ['0.4', '0.3', '0.7', '0.8', '0.5'],
 ['0.2', '0.4', '0.3', '0.7', '0.6'],
 ['0.6', '0.55', '0.7', '0.65', '0.2', '0.6', '0.55', '0.7', '0.65', '0.2'],
 ['0.2', '0.4', '0.7', '0.8', '0.6'],
 ['0.45', '0.75', '0.3', '0.6', '0.35'],
 ['0.1', '0.2', '0.15', '0.2', '0.7'],
 ['0.2', '0.5', '0.3', '0.7', '0.6'],
 ['0.2', '0.7', '0.8', '0.9', 

In [94]:
FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv",
                     index_col=0)
FI_test['label'] = FI_test[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_test['label'] = FI_test['label'].str.split(' ')

FI_test

,id,Script,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness,token_len,user,l_Script,label
0,htH89DBizno.004.mp4,다른 방에서 미친 듯이 달려가서 없는지 확인했는데 없는 것 같았어요 이제 잠을 충분...,0.485981,0.645833,0.681319,0.669903,0.626168,0.822222,36,htH89DBiznomp4,going nuts from another room run in there to c...,"[0.8222222222222221, 0.6699029126213591, 0.485..."
1,p_wf-KszNlk.001.mp4,갈 길이 조금 남았지만 당신이 필요해서 Q&A 비디오를하기로 결정했습니다. 분명히 ...,0.616822,0.593750,0.692308,0.514563,0.570093,0.655556,40,p_wf-KszNlkmp4,i've got a little bit to go but we need you th...,"[0.6555555555555556, 0.5145631067961165, 0.616..."
2,MuYYY3XaJ7Q.001.mp4,영상은 양보다 질이 중요하기 때문에 모든 사람이 볼 수 있는 기회를 제공하기 위해 ...,0.467290,0.625000,0.560440,0.524272,0.514019,0.522222,50,MuYYY3XaJ7Qmp4,a video's quality over quantity so everyone ca...,"[0.5222222222222223, 0.5242718446601942, 0.467..."
3,0MB91ku0eEw.005.mp4,"제가 한 일은 온라인에서 사람들을 가상으로 만났을 때 그들이 게시하는 것을보고 ""저...",0.411215,0.458333,0.714286,0.660194,0.570093,0.400000,53,0MB91ku0eEwmp4,you really want to that's made things a lot ea...,"[0.4, 0.6601941747572816, 0.4112149532710281, ..."
4,WpEZOSrENL0.003.mp4,이 연습의 요점은 실제로 답을 찾는 것이 아닙니다. 요점은 단순히 이러한 질문을 스...,0.317757,0.437500,0.384615,0.524272,0.448598,0.411111,39,WpEZOSrENL0mp4,that's perfectly okay the point isn't actually...,"[0.4111111111111111, 0.5242718446601942, 0.317..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1965,NzCflfxgvwo.001.mp4,두두두 아니 그게 내가이 비디오를 만든 이유가 아니야 나는이 비디오를 만들었다 나는...,0.289720,0.312500,0.208791,0.135922,0.177570,0.300000,49,NzCflfxgvwomp4,du du du no that's not why i made this video i...,"[0.3, 0.1359223300970873, 0.2897196261682243, ..."
1966,ch2BcBv4SdQ.003.mp4,그들은 항상 그렇게 합니다.,0.719626,0.781250,0.670330,0.572816,0.728972,0.722222,6,ch2BcBv4SdQmp4,they do it all the time,"[0.7222222222222221, 0.5728155339805825, 0.719..."
1967,It0-yCwOc5k.001.mp4,편안하고 누구도 불편함을 느끼지 않기를 바랍니다 그래서 내 채널에서 다른 사람들을 ...,0.355140,0.395833,0.472527,0.446602,0.411215,0.677778,30,It0-yCwOc5kmp4,comfortable and i don't want anyone to feel un...,"[0.6777777777777777, 0.4466019417475728, 0.355..."
1968,1Wh2mIFrGvk.000.mp4,새로운 근육을 만들기에 충분한 칼로리를 공급하지 못하고 있습니다 새로운 조직은 어디...,0.467290,0.645833,0.527473,0.669903,0.560748,0.622222,44,1Wh2mIFrGvkmp4,you're not giving yourself enough calories to ...,"[0.6222222222222222, 0.6699029126213591, 0.467..."


In [95]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [96]:
error_idx

[]

In [97]:
np.array(MAE_arr).mean(axis=0)

array([0.7756677 , 0.777749  , 0.80246082, 0.81028451, 0.72891629])

In [99]:
np.mean([0.7756677 , 0.777749  , 0.80246082, 0.81028451, 0.72891629])

0.7790156639999999

### GPT4 Inference - One Shot

In [100]:
key_path = "/home/egg2018037024/juphome/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

one_shot_text = "현재 가장 좋아하는 책 사실 몇 달 전에 서점에서 이 책을 샀는데 읽을 기회가 없었는데 읽기 시작했어요 한번 시작하면 내려놓고 싶지 않은 책 조이스 마이어의 <용감하게 살기>라는 책이에요."

for data in tqdm(FI_test['Script'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + one_shot_text + '",\n Results- [O: 0.6333, C: 0.7184, E: 0.4019, A: 0.4176, N: 0.5313] \n'
            },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages,
            temperature=0.9,
        #    top_p = 0.92
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

100%|██████████| 1970/1970 [1:18:26<00:00,  2.39s/it]


In [102]:
result_df['Inference'].to_list()

['[O: 0.2150, C: 0.3200, E: 0.2900, A: 0.2100, N: 0.7700]',
 '[O: 0.2700, C: 0.3200, E: 0.5900, A: 0.2300, N: 0.8000]',
 '[O: 0.6110, C: 0.7290, E: 0.4700, A: 0.7050, N: 0.2280]',
 '[O: 0.410, C: 0.610, E: 0.670, A: 0.480, N: 0.320]',
 '[O: 0.7, C: 0.6, E: 0.2, A: 0.6, N: 0.3] \n\nExplanation:\n- Openness (O): The sentence suggests curiosity and introspection, reflecting a tendency to engage in deep thought and contemplation, indicative of a higher level of openness.\n- Conscientiousness (C): The structured approach to self-exploration suggests some level of conscientiousness, though it is more about exploration than organization or diligence.\n- Extraversion (E): The content is introspective and inward-looking, which generally aligns more with introversion; thus, extraversion is lower.\n- Agreeableness (A): The tone is non-confrontational and gentle, indicating a moderate level of agreeableness.\n- Neuroticism (N): The sentence does not explicitly show significant anxiety or emotional

In [103]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [104]:
score

[['0.2150', '0.3200', '0.2900', '0.2100', '0.7700'],
 ['0.2700', '0.3200', '0.5900', '0.2300', '0.8000'],
 ['0.6110', '0.7290', '0.4700', '0.7050', '0.2280'],
 ['0.410', '0.610', '0.670', '0.480', '0.320'],
 ['0.7', '0.6', '0.2', '0.6', '0.3'],
 ['0.3077', '0.4191', '0.2658', '0.5632', '0.4924'],
 ['0.6010', '0.3820', '0.7890', '0.8450', '0.2200'],
 ['0.7000', '0.8200', '0.3000', '0.4000', '0.6000'],
 ['0.45', '0.20', '0.30', '0.25', '0.70'],
 ['0.3210', '0.2308', '0.1472', '0.3085', '0.7823'],
 ['0.4113', '0.3297', '0.5610', '0.8546', '0.4532'],
 ['0.6571', '0.2419', '0.7890', '0.5320', '0.3045'],
 ['0.4520', '0.5871', '0.6094', '0.7432', '0.4819'],
 ['0.3600', '0.5320', '0.6540', '0.4800', '0.2500'],
 ['0.2581', '0.1537', '0.5662', '0.1996', '0.7224'],
 ['0.412', '0.239', '0.692', '0.587', '0.508'],
 ['0.3114', '0.5689', '0.3246', '0.4803', '0.4275'],
 ['0.2100', '0.5500', '0.7500', '0.8500', '0.1500'],
 ['0.3210', '0.2310', '0.7234', '0.6450', '0.6875'],
 ['0.312', '0.648', '0.239',

In [105]:
FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv",
                     index_col=0)
FI_test['label'] = FI_test[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_test['label'] = FI_test['label'].str.split(' ')

FI_test

,id,Script,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness,token_len,user,l_Script,label
0,htH89DBizno.004.mp4,다른 방에서 미친 듯이 달려가서 없는지 확인했는데 없는 것 같았어요 이제 잠을 충분...,0.485981,0.645833,0.681319,0.669903,0.626168,0.822222,36,htH89DBiznomp4,going nuts from another room run in there to c...,"[0.8222222222222221, 0.6699029126213591, 0.485..."
1,p_wf-KszNlk.001.mp4,갈 길이 조금 남았지만 당신이 필요해서 Q&A 비디오를하기로 결정했습니다. 분명히 ...,0.616822,0.593750,0.692308,0.514563,0.570093,0.655556,40,p_wf-KszNlkmp4,i've got a little bit to go but we need you th...,"[0.6555555555555556, 0.5145631067961165, 0.616..."
2,MuYYY3XaJ7Q.001.mp4,영상은 양보다 질이 중요하기 때문에 모든 사람이 볼 수 있는 기회를 제공하기 위해 ...,0.467290,0.625000,0.560440,0.524272,0.514019,0.522222,50,MuYYY3XaJ7Qmp4,a video's quality over quantity so everyone ca...,"[0.5222222222222223, 0.5242718446601942, 0.467..."
3,0MB91ku0eEw.005.mp4,"제가 한 일은 온라인에서 사람들을 가상으로 만났을 때 그들이 게시하는 것을보고 ""저...",0.411215,0.458333,0.714286,0.660194,0.570093,0.400000,53,0MB91ku0eEwmp4,you really want to that's made things a lot ea...,"[0.4, 0.6601941747572816, 0.4112149532710281, ..."
4,WpEZOSrENL0.003.mp4,이 연습의 요점은 실제로 답을 찾는 것이 아닙니다. 요점은 단순히 이러한 질문을 스...,0.317757,0.437500,0.384615,0.524272,0.448598,0.411111,39,WpEZOSrENL0mp4,that's perfectly okay the point isn't actually...,"[0.4111111111111111, 0.5242718446601942, 0.317..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1965,NzCflfxgvwo.001.mp4,두두두 아니 그게 내가이 비디오를 만든 이유가 아니야 나는이 비디오를 만들었다 나는...,0.289720,0.312500,0.208791,0.135922,0.177570,0.300000,49,NzCflfxgvwomp4,du du du no that's not why i made this video i...,"[0.3, 0.1359223300970873, 0.2897196261682243, ..."
1966,ch2BcBv4SdQ.003.mp4,그들은 항상 그렇게 합니다.,0.719626,0.781250,0.670330,0.572816,0.728972,0.722222,6,ch2BcBv4SdQmp4,they do it all the time,"[0.7222222222222221, 0.5728155339805825, 0.719..."
1967,It0-yCwOc5k.001.mp4,편안하고 누구도 불편함을 느끼지 않기를 바랍니다 그래서 내 채널에서 다른 사람들을 ...,0.355140,0.395833,0.472527,0.446602,0.411215,0.677778,30,It0-yCwOc5kmp4,comfortable and i don't want anyone to feel un...,"[0.6777777777777777, 0.4466019417475728, 0.355..."
1968,1Wh2mIFrGvk.000.mp4,새로운 근육을 만들기에 충분한 칼로리를 공급하지 못하고 있습니다 새로운 조직은 어디...,0.467290,0.645833,0.527473,0.669903,0.560748,0.622222,44,1Wh2mIFrGvkmp4,you're not giving yourself enough calories to ...,"[0.6222222222222222, 0.6699029126213591, 0.467..."


In [106]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [107]:
error_idx

[]

In [108]:
np.array(MAE_arr).mean(axis=0)

array([0.78392659, 0.80709438, 0.82091478, 0.82104526, 0.76704043])

In [109]:
np.mean([0.78392659, 0.80709438, 0.82091478, 0.82104526, 0.76704043])

0.800004288

### GPT4 Inference - Five Shot

In [4]:
key_path = "/home/egg2018037024/juphome/gpt3.5_key.json"

with open(key_path, 'r') as f:
    key_json = json.load(f)
    openai_client = OpenAI(api_key=key_json['key'])    


FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv")
result_df = pd.DataFrame(columns=["Text", "Inference"])

few_shot_text0 = "현재 가장 좋아하는 책 사실 몇 달 전에 서점에서 이 책을 샀는데 읽을 기회가 없었는데 읽기 시작했어요 한번 시작하면 내려놓고 싶지 않은 책 조이스 마이어의 <용감하게 살기>라는 책이에요."
few_shot_text1 = "제가 타겟에서 일했던 곳과 월마트에서 일했던 곳 제가 유튜브를하지 않았다면 아마 여전히 그 일을하고 있지 않았을 것입니다. 저는 항상 손으로 일하고 물건을 만드는 데 관심이 있었지만 건설은 그랬어요"
few_shot_text2 = "나쁘지만 그들이 만든 최악의 영화는 아닙니다. 그들이 만든 최악의 영화는 괴물이라고 생각합니다. 괴물은 텍사스의 비료 판매원이 만들었다는 점에서 정말 흥미 롭습니다."
few_shot_text3 = "매우 소외감을 느낀다는 것은 확실히 제 자존감과 제 자신을 보는 방식에 타격을 주었습니다 제가 이것을 극복한 방법은 제가 매우 긍정적인 환경에 있는지 확인했습니다 저는 합창단처럼 제가 좋아하는 일을 계속하고 예술을 만들었습니다."
few_shot_text4 = "감정을 읽는 것에 대해 사람들의 얼굴을 보고 그들의 감정을 어떻게 알 수 있는지에 관한 것입니다 흥미롭습니다 나는 유령 요금소를 읽을 것입니다 나는 그것을 읽지 않았다 나는 많은 사람들이 가지고 있다는 것을 알고 있습니다 미안합니다 나는 그것을 읽지 않았다."


for data in tqdm(FI_test['Script'].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n",
        },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text0 + '",\n Results- [O: 0.6333, C: 0.7184, E: 0.4019, A: 0.4176, N: 0.5313] \n'
            },
       {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text1 + '",\n Results- [O: 0.6889, C: 0.5631, E: 0.5981, A: 0.6374, N: 0.6563] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text2 + '",\n Results- [O: 0.7778, C: 0.4175, E: 0.5794, A: 0.6813, N: 0.65625] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text3 + '",\n Results- [O: 0.9000, C: 0.8835, E: 0.7103, A: 0.7912, N: 0.8229] \n'
            },
        {
                "role": "assistant",
                "content": 'Given Sentence: "' + few_shot_text4 + '",\n Results- [O: 0.3889, C: 0.4660, E: 0.2056, A: 0.4176, N: 0.3646] \n'
            },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]

    response = openai_client.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages,
            temperature=0.9,
        #    top_p = 0.92
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

100%|██████████| 1970/1970 [2:37:44<00:00,  4.80s/it]  


In [6]:
result_df['Inference'].to_list()

['Given Sentence: "다른 방에서 미친 듯이 달려가서 없는지 확인했는데 없는 것 같았어요 이제 잠을 충분히 못 자서 가위를 어딘가로 옮겼을 수도 있지만 맹세컨대",\nResults- [O: 0.3111, C: 0.5049, E: 0.5140, A: 0.5055, N: 0.7604]',
 'Given Sentence: "갈 길이 조금 남았지만 당신이 필요해서 Q&A 비디오를하기로 결정했습니다. 분명히 당신들의 도움이 필요하거나 여자들 젠장 이제 나는 죽게 될 것입니다.",\nResults- [O: 0.4811, C: 0.3364, E: 0.5732, A: 0.2389, N: 0.8217]',
 '[O: 0.6211, C: 0.7533, E: 0.5789, A: 0.7450, N: 0.2381]',
 '[O: 0.6289, C: 0.7263, E: 0.6841, A: 0.5472, N: 0.3125]',
 'Given Sentence: "이 연습의 요점은 실제로 답을 찾는 것이 아닙니다. 요점은 단순히 이러한 질문을 스스로에게 한 다음 관찰하는 것입니다. 답이 나타나지 않는 경우 답이 나타나는지 관찰하십시오.",\n Results- [O: 0.75, C: 0.3, E: 0.2, A: 0.55, N: 0.4]',
 'Given Sentence: "시력이 정말 안 좋았어요 그녀는 당신이 할 수 있고 읽을 수 있다고 말했고 저는 그것을했고 지금까지도 그것을 좋아합니다. 제가 실제로 책을 처음 읽기 시작한 것은 14 년 전이었습니다. 저는 항상 책을 좋아했기 때문에 그렇습니다.",\nResults- [O: 0.6750, C: 0.7444, E: 0.4830, A: 0.6923, N: 0.2292]',
 'Given Sentence: "여러분에게 물어보거나 이야기하고 싶은 것은 여러분들이 저에게 어떤 종류의 동영상을 보고 싶어 하나요? 외모나 패션 영상, 헤어 튜토리얼에 대한 구체적인 요청이 있거나 더 많은 걸 토크 영상에 대한 요청이 있는 경우",\n Res

In [7]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [8]:
score

[['0.3111', '0.5049', '0.5140', '0.5055', '0.7604'],
 ['0.4811', '0.3364', '0.5732', '0.2389', '0.8217'],
 ['0.6211', '0.7533', '0.5789', '0.7450', '0.2381'],
 ['0.6289', '0.7263', '0.6841', '0.5472', '0.3125'],
 ['0.75', '0.3', '0.2', '0.55', '0.4'],
 ['0.6750', '0.7444', '0.4830', '0.6923', '0.2292'],
 ['0.7322', '0.5214', '0.8689', '0.8128', '0.2311'],
 ['0.7111', '0.7379', '0.2710', '0.5055', '0.5729'],
 ['0.6111', '0.3495', '0.2523', '0.4505', '0.7083'],
 ['0.4556', '0.4951', '0.4019', '0.3297', '0.7188'],
 ['0.5200', '0.4400', '0.7600', '0.8900', '0.3100'],
 ['0.6222', '0.3495', '0.6449', '0.6484', '0.2396'],
 ['0.5100', '0.7200', '0.6300', '0.8100', '0.4900'],
 ['0.6456', '0.5322', '0.7684', '0.6248', '0.2917'],
 ['0.3611', '0.2136', '0.4495', '0.3077', '0.7188'],
 ['0.4111', '0.1942', '0.6822', '0.7252', '0.4688'],
 ['0.3000', '0.5629', '0.2523', '0.5934', '0.6979'],
 ['0.4556', '0.5049', '0.7664', '0.8242', '0.1875'],
 ['0.3211', '0.4122', '0.7128', '0.7704', '0.6579'],
 ['0.5

In [9]:
FI_test = pd.read_csv("/mnt/HDD8TB/Data/PAI/FI/test_ko_pp.csv",
                     index_col=0)
FI_test['label'] = FI_test[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_test['label'] = FI_test['label'].str.split(' ')

FI_test

,id,Script,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness,token_len,user,l_Script,label
0,htH89DBizno.004.mp4,다른 방에서 미친 듯이 달려가서 없는지 확인했는데 없는 것 같았어요 이제 잠을 충분...,0.485981,0.645833,0.681319,0.669903,0.626168,0.822222,36,htH89DBiznomp4,going nuts from another room run in there to c...,"[0.8222222222222221, 0.6699029126213591, 0.485..."
1,p_wf-KszNlk.001.mp4,갈 길이 조금 남았지만 당신이 필요해서 Q&A 비디오를하기로 결정했습니다. 분명히 ...,0.616822,0.593750,0.692308,0.514563,0.570093,0.655556,40,p_wf-KszNlkmp4,i've got a little bit to go but we need you th...,"[0.6555555555555556, 0.5145631067961165, 0.616..."
2,MuYYY3XaJ7Q.001.mp4,영상은 양보다 질이 중요하기 때문에 모든 사람이 볼 수 있는 기회를 제공하기 위해 ...,0.467290,0.625000,0.560440,0.524272,0.514019,0.522222,50,MuYYY3XaJ7Qmp4,a video's quality over quantity so everyone ca...,"[0.5222222222222223, 0.5242718446601942, 0.467..."
3,0MB91ku0eEw.005.mp4,"제가 한 일은 온라인에서 사람들을 가상으로 만났을 때 그들이 게시하는 것을보고 ""저...",0.411215,0.458333,0.714286,0.660194,0.570093,0.400000,53,0MB91ku0eEwmp4,you really want to that's made things a lot ea...,"[0.4, 0.6601941747572816, 0.4112149532710281, ..."
4,WpEZOSrENL0.003.mp4,이 연습의 요점은 실제로 답을 찾는 것이 아닙니다. 요점은 단순히 이러한 질문을 스...,0.317757,0.437500,0.384615,0.524272,0.448598,0.411111,39,WpEZOSrENL0mp4,that's perfectly okay the point isn't actually...,"[0.4111111111111111, 0.5242718446601942, 0.317..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1965,NzCflfxgvwo.001.mp4,두두두 아니 그게 내가이 비디오를 만든 이유가 아니야 나는이 비디오를 만들었다 나는...,0.289720,0.312500,0.208791,0.135922,0.177570,0.300000,49,NzCflfxgvwomp4,du du du no that's not why i made this video i...,"[0.3, 0.1359223300970873, 0.2897196261682243, ..."
1966,ch2BcBv4SdQ.003.mp4,그들은 항상 그렇게 합니다.,0.719626,0.781250,0.670330,0.572816,0.728972,0.722222,6,ch2BcBv4SdQmp4,they do it all the time,"[0.7222222222222221, 0.5728155339805825, 0.719..."
1967,It0-yCwOc5k.001.mp4,편안하고 누구도 불편함을 느끼지 않기를 바랍니다 그래서 내 채널에서 다른 사람들을 ...,0.355140,0.395833,0.472527,0.446602,0.411215,0.677778,30,It0-yCwOc5kmp4,comfortable and i don't want anyone to feel un...,"[0.6777777777777777, 0.4466019417475728, 0.355..."
1968,1Wh2mIFrGvk.000.mp4,새로운 근육을 만들기에 충분한 칼로리를 공급하지 못하고 있습니다 새로운 조직은 어디...,0.467290,0.645833,0.527473,0.669903,0.560748,0.622222,44,1Wh2mIFrGvkmp4,you're not giving yourself enough calories to ...,"[0.6222222222222222, 0.6699029126213591, 0.467..."


In [10]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [11]:
error_idx

[]

In [75]:
re_df = pd.DataFrame(columns=["Text", "Inference"])

for data in tqdm(FI_test['Script'][error_idx].to_list()):
    user = data
    messages = [
        {
        "role": "system",
        "content":"You are an expert in analyzing sentences for the OCEAN personality traits. \
        The OCEAN personality traits assess the following: O (Openness), C (Conscientiousness), E (Extraversion), A (Agreeableness), and N (Neuroticism). \
        Read the given sentence and analyze the extent to which each of the OCEAN personality traits is exhibited in the sentence. \
        Assign a score between 0 and 1 for each trait based on how strongly the trait is represented in the sentence. \
        A score close to 0 means the trait is not represented at all, while a score close to 1 means the trait is very strongly represented. \
        You should provide the output as the scores for the five OCEAN personality traits for the given sentence, similar to the example below. \
        Example: [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323]",
        },
        {
        "role": "user",
        "content": 'Given Sentence: "' + user + '",\n Results- [O:'
        }
    ]
    
    response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.9,
        #    top_p = 0.92
            max_tokens=1000,
    )
    out_text = response.choices[0].message.content.strip()
    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    re_df = pd.concat([re_df, tmp_df])

100%|██████████| 1/1 [00:03<00:00,  3.98s/it]


In [81]:
re_df

,Text,Inference
0,보통 편집이 많이 걸리는 당일에 보통 꽤 오래 걸립니다 크리스마스처럼 마지막으로 한...,"[O: 0.1634, C: 0.7501, E: 0.3865, A: 0.5887, N..."


In [82]:
re_score = []
p = re.compile('0[.]\d*')

for content in re_df['Inference'].to_list():
    tmp = p.findall(content)
    re_score.append(tmp)

In [83]:
re_score

[['0.1634', '0.7501', '0.3865', '0.5887', '0.4219']]

In [84]:
for i, idx in enumerate(error_idx):
    score[idx] = re_score[i]

In [85]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [12]:
np.array(MAE_arr).mean(axis=0)

array([0.80206517, 0.82098391, 0.81051178, 0.82531748, 0.75400396])

In [14]:
np.mean([0.80206517, 0.82098391, 0.81051178, 0.82531748, 0.75400396])

0.80257646